## Librerie

In [1]:
from scipy.optimize import minimize, basinhopping
import numpy as np
import pandas as pd

## Parametri problema

In [2]:
# Consumi famiglia 1, famiglia 2, ...
consumi = np.array([1500, 5700, 3400, 300])

# Produzioni ricetta1, ricetta2, ...
produzioni = np.array([3000, 6300, 1200])

# Ricette
#           | Ricetta1 | Ricetta2 | ...
# --------------------------------------
# Famiglia1 |          |          |
# Famiglia2 |          |          |
# ...
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

# Composizioni ricette per famiglia
#           | Materiale1 | Materiale2 | ...
# --------------------------------------
# Famiglia1 |            |            |
# Famiglia2 |            |            |
# ...
composizioni_famiglia = np.array([
    0.58, 0.42, 0,
    1, 0, 0,
    0, 1, 0,
    0, 0, 1
])

# Range ammissibile percentuale materiale per ricetta
#           | Materiale1        | Materiale2 | ...
# ------------------------------------------------
# Ricetta1  | (val att. ,range) |            |
# Ricetta2  |                   |            |
# ...
range_ric_mat = np.array([
    [(0.58, 0.01), (0.396, 0.003), (0.024, 0.001)], # ricetta 0
    [(0.625, 0.005), (None, None), (None, None)],
    [(0.62, 0.01), (None, None), (None, None)],
])

In [3]:
%run ./importazione.ipynb

(10,)
(28,)
(28, 10)
(28, 13)
(10, 13)
(10, 13)
(10, 13)


In [4]:
print('Verifica dimensioni matrici')
print(f'Consumi (num. famiglie)): {consumi.shape}')
print(f'Produzioni (num. ricette): {produzioni.shape}')
print(f'Ricette (famiglie x ricette): {ricette.shape}')
print(f'Composizioni (famiglie x materiali): {composizioni_famiglia.shape}')
print(f'Range (ricette x materiali): {range_ric_mat.shape}')

Verifica dimensioni matrici
Consumi (num. famiglie)): (28,)
Produzioni (num. ricette): (10,)
Ricette (famiglie x ricette): (280,)
Composizioni (famiglie x materiali): (364,)
Range (ricette x materiali): (10, 13)


## Accrocchio per compattare ricetta

In [5]:
POSITIONS_COMP = []
RICETTA_COMP = []

for index, value in enumerate(ricette):
    if value != 0:
        POSITIONS_COMP.append(index)
        RICETTA_COMP.append(value)

def rebuild_ricetta(ricetta_comp):
    rebuild = np.zeros(len(ricette))
    for index, pos in enumerate(POSITIONS_COMP):
        rebuild[pos] = ricetta_comp[index]
    return rebuild

print(rebuild_ricetta(RICETTA_COMP)[0:20])
print(ricette[0:20])
print(len(RICETTA_COMP))

[0.    0.029 0.    0.    0.    0.    0.    0.    0.    0.    0.455 0.48
 0.03  0.475 0.    0.    0.    0.    0.005 0.   ]
[0.    0.029 0.    0.    0.    0.    0.    0.    0.    0.    0.455 0.48
 0.03  0.475 0.    0.    0.    0.    0.005 0.   ]
76


## Calcolo resa globale

In [6]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0389346559077535)'

## Funzioni di calcolo

In [7]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    ricetta = rebuild_ricetta(ricetta) #REBUILD
    return ricetta.reshape(-1, len(produzioni)) * produzioni #/ resa_globale

In [8]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return np.sum(matrice_consumi, axis=1)

In [9]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sqrt(np.sum(np.square(tot_consumi-consumi)))/1000
    return tot_err

In [10]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return np.sum(matrice_consumi, axis=0)/produzioni

In [11]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum(np.square(tot_resa - resa_globale))

In [12]:
# Percentuali di ciascun materiale in una ricetta
def perc_mat(ricetta, id_ric):
    cons_fam = calc_mat_consumi(ricetta)
    compos = composizioni_famiglia.reshape(len(consumi), -1)
    cons_ricetta0 = np.vstack(cons_fam[:,id_ric])
    cons_materiali_ricetta0 = cons_ricetta0 * compos
    tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
    tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0) if np.sum(tot_cons_ricetta0) != 0 else np.zeros((len(tot_cons_ricetta0),1))
    return tot_perc_ricetta0

In [13]:
# Funzione errore percentuale materiale (obiettivo: >=0)
def err_perc_mat(ricetta, id_ric, id_mat, expected_val, expected_error):
    return expected_error - np.abs(perc_mat(ricetta, id_ric=id_ric)[id_mat] - expected_val)

## Ottimizzazione

In [51]:
%%time

constraints = [
    {'type': 'eq', 'fun': calc_err_totali},
]

for id_ric, ric in enumerate(range_ric_mat):
    for id_mat, mat in enumerate(ric):
        if not np.isnan(mat[0]):
            constr = {'type': 'ineq', 'fun': err_perc_mat, 'args': (id_ric, id_mat, mat[0], mat[1])}
            constraints.append(constr)

# constraints = (
#     {'type': 'eq', 'fun': calc_err_totali},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 0, 0.58, 0.01)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 1, 0.396, 0.003)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 2, 0.024, 0.001)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (1, 0, 0.625, 0.005)},
#     # {'type': 'ineq', 'fun': err_perc_mat, 'args': (2, 0, 0.62, 0.01)},
# )

#DEBUG
print(len(constraints))
constraints = constraints[0] #DEBUG 40 ok, oltre boom

bounds = list(( (0, 1) for x in range(len(RICETTA_COMP) )))

count_iterations = 0
def callback(x, y=None):
    global count_iterations
    print(f'{count_iterations} - {calc_err_totali(x)}')
    count_iterations += 1

# Minimizzazione con TRUST-CONSTR
res = minimize(
    calc_error_resa, 
    RICETTA_COMP, 
    # method='trust-constr', 
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    callback=callback,
    options={'disp': True, 'maxiter':200}
)

# Basin hopping con TRUST_CONSTR
# res = basinhopping(
    
#     calc_error_resa, 
#     RICETTA_COMP,
    
#     stepsize=0.05,
#     niter=50,
    
#     minimizer_kwargs = {
#         "method":'trust-constr',
#         "constraints":constraints,
#         "bounds":bounds,
#         "callback":callback,
#         "options":{'disp': True, 'maxiter':200}
#     }
    
# )


res.x = res.x / resa_globale

print(res)

# res = basinhopping(
#     calc_error_resa, 
#     RICETTA_COMP,
#     stepsize=0.05,
#     niter=10,
    
#     minimizer_kwargs = {
#         "method":'SLSQP',
#         "constraints":constraints,
#         "bounds":bounds,
#         "callback":callback,
#         "options":{'disp': True, 'maxiter':20}
#     }
# )
# print(res)

# for i in range(10):
    
#     res = minimize(
#         calc_error_resa, 
#         res.x, 
#         method='trust-constr', 
#         # method='SLSQP',
#         constraints=constraints,
#         bounds=bounds,
#         options={'disp': True, 'maxiter':10}
#     )
        
#     res = minimize(
#         calc_error_resa, 
#         res.x,
#         method='SLSQP',
#         constraints=constraints,
#         bounds=bounds,
#         callback=callback,
#         options={'disp': True, 'maxiter':10}
#     )

# print(res)


131
0 - 44619.16825186107
1 - 5432.744497164486
2 - 7146.084231163501
3 - 4099.254219960092
4 - 19970.679917506634
5 - 5213.941946237777
6 - 4877.855310866015
7 - 9093.724971844746
8 - 11566.827931132302
9 - 7955.259532860104
10 - 10574.968854950788
11 - 4125.612111756258
12 - 5534.864996187272
13 - 5845.17305862772
14 - 5983.768391080164
15 - 2290.816075039431
16 - 2351.4132198743478
17 - 4488.856547158872
18 - 2280.721841850267
19 - 6843.533290771178
20 - 2399.6561255958823
21 - 3101.2764908924473
22 - 2628.3132839206037
23 - 4924.398180626395
24 - 1220.1763440866105
25 - 1379.3897225369842
26 - 1765.746800652831
27 - 1228.2265930557166
28 - 1272.8491924546322
29 - 1744.1488746348705
30 - 908.5505929725388
31 - 724.7214616686209
32 - 577.5381965046287
33 - 466.9388198546929
34 - 267.4681028657259
35 - 264.50672573439823
36 - 210.464458465868
37 - 299.20758759065654
38 - 183.90777077177583
39 - 115.16371555056739
40 - 91.4606730815152
41 - 105.7087716634032
42 - 83.5952490532919
43 - 

In [52]:
# Esperimento
# Proviamo a escludere un constraint alla volta e vediamo se troviamo una situazione
# in cui il problema converge.

# for i in range(1, len(constraints)):
    
#     print(i)
    
#     const_red = constraints[0:i] + constraints[i+1:]

#     res = minimize(
#         calc_error_resa, 
#         ricette, 
#         method='SLSQP',
#         constraints=const_red,
#         bounds=bounds,
#         options={'disp': False, 'maxiter':100}
#     )

#     print(f'{res.success} - {res.message} ({res.nit} iterations)')


## Verifiche

In [53]:
pd.options.display.float_format = '{:.2f}'.format

In [54]:
# res.x = res.x / resa_globale
result = rebuild_ricetta(res.x)

In [55]:
print("Percentuali aggiustate (in %)")
print(np.sum(result.reshape(len(consumi), len(produzioni))*100, axis=0))
pd.DataFrame(
    result.reshape(len(consumi), len(produzioni))*100
)

Percentuali aggiustate (in %)
[100.00024291 100.0010556  100.00037941  99.9995162  100.00019358
 100.00003971  99.99999418 100.0000673  100.000288   100.00000083]


,0,1,2,3,4,5,6,7,8,9
0,0.00,3.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,46.15,53.24,5.56,53.09,0.00,0.00,0.00,0.00,11.34,0.00
2,0.00,0.00,36.96,0.00,7.07,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,11.75,0.00,0.00,0.00,22.07,0.00
4,0.00,0.00,0.00,0.00,0.00,4.98,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,44.03,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.96
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.46,0.00
8,27.22,24.86,5.36,30.82,0.00,0.00,0.00,0.00,11.20,0.00
9,0.00,0.00,26.55,0.00,3.14,14.82,0.00,0.00,0.00,0.00


In [56]:
print("Matrice consumi")
pd.DataFrame(
    calc_mat_consumi(res.x)
)

Matrice consumi


,0,1,2,3,4,5,6,7,8,9
0,0.00,1200635.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2675828.40,18631914.11,416258.33,49503377.82,0.00,0.00,0.00,0.00,568558.30,0.00
2,0.00,0.00,2764548.29,0.00,265408.86,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,440768.95,0.00,0.00,0.00,1106629.35,0.00
4,0.00,0.00,0.00,0.00,0.00,67629.67,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,691358.85,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22740.91
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,323618.85,0.00
8,1578434.18,8699282.68,401019.64,28737094.20,0.00,0.00,0.00,0.00,561715.91,0.00
9,0.00,0.00,1986008.44,0.00,117848.10,201282.11,0.00,0.00,0.00,0.00


In [57]:
print('Verifica totale consumi')
pd.DataFrame(
    [calc_tot_consumi(calc_mat_consumi(res.x))]
)

Verifica totale consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1200635.14,71795936.97,3029957.15,1547398.30,67629.67,691358.85,22740.91,323618.85,39977546.61,2305138.65,...,10020078.23,1430529.07,55712.32,2262.00,4274.77,11746.94,23414.65,204.49,6155.01,10436.41


In [58]:
print('Verifica errore consumi')
pd.DataFrame([calc_tot_consumi(calc_mat_consumi(res.x))*resa_globale - consumi])

Verifica errore consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-0.55,-0.66,-0.39,-0.88,-0.69,-0.63,-0.68,-1.16,-0.70,-0.57,...,-0.48,-0.77,-0.54,0.07,-0.79,-0.69,-0.71,-0.55,-0.84,-0.25


In [59]:
# np.sum(calc_mat_consumi(res.x)*resa_globale, axis=1) / consumi * 100

In [60]:
print(f'Verifica rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x))]
)

Verifica rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [61]:
print(f'Verifica errore rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale]
)

Verifica errore rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04


### Errori percentuali materiali

In [62]:
def print_err_mat(id_ricetta, ricetta):
    return pd.DataFrame([
        perc_mat(RICETTA_COMP, id_ricetta) * 100,
        map(lambda x: x[0]*100, list(range_ric_mat[id_ricetta])),
        perc_mat(ricetta, id_ricetta) * 100
    ])

In [63]:
print('Percentuali materiali ricetta 0')
print_err_mat(0, res.x)

Percentuali materiali ricetta 0


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,55.65,13.37,0.12,0.14,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,30.67
1,59.40,2.05,0.16,0.15,0.05,0.02,0.01,0.00,0.01,0.01,0.00,0.00,38.13
2,58.58,8.58,0.12,0.14,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,32.53


In [64]:
print('Percentuali materiali ricetta 1')
print_err_mat(1, res.x)

Percentuali materiali ricetta 1


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,58.11,11.74,0.14,0.16,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,29.79
1,58.00,2.90,0.16,0.16,0.10,0.02,0.01,0.00,0.01,0.01,0.00,0.00,38.62
2,59.07,4.42,0.15,0.17,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,36.12


In [65]:
print('Percentuali materiali ricetta 2')
print_err_mat(2, res.x)

Percentuali materiali ricetta 2


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,64.22,0.81,0.08,0.07,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,34.79
1,61.80,2.00,0.06,0.06,0.05,0.02,0.01,0.00,0.10,0.01,0.00,0.00,35.88
2,62.74,0.88,0.08,0.08,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,36.20


### Funzioni errore

In [66]:
for index,c in enumerate(constraints):
    # print(c)
    if 'args' in c:
        print(f"{index} - {c['fun'](res.x, *c['args']):.4f} (ric:{c['args'][0]} mat:{c['args'][1]})")
    else:
        print(c['fun'](res.x))

TypeError: string indices must be integers, not 'str'

# ?

In [ ]:
r = rebuild_ricetta(res.x).reshape(len(consumi), len(produzioni))
print(np.sum(r, axis=0))
print(np.sum(r*produzioni, axis=1))
print(np.sum(r*produzioni, axis=1) - consumi)
print(np.sum(np.sum(r*produzioni, axis=1) - consumi))

In [ ]:
#pd.DataFrame(rebuild_ricetta(res.x)).to_csv('trust_constr_5k_iter.csv')